In [1]:
print("Hallo Welt")

Hallo Welt


In [2]:
%pip install whylogs[whylabs]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 14.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.7/370.7 kB 62.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
%%writefile predict.py
import os
import joblib
import pandas as pd
import numpy as np
import whylogs as why

logger = why.logger(mode="rolling", interval=15, when="M",
                                base_name="profile_name")
logger.append_writer("whylabs")


def predict_fn(data, model):
    pandas_df = pd.DataFrame(data)
    pandas_df["predicted_output"] = model.predict(data)
    logger.log(pandas_df)
    return np.array(pandas_df["predicted_output"].values)

def model_fn(model_dir):
    """Deserialized and return fitted model

    Note that this should have the same name as the serialized model in the main method
    """
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

Overwriting predict.py


In [48]:
%%writefile train.py
from __future__ import print_function

import argparse
import joblib
import os
import pandas as pd

from sklearn import tree
from sklearn.datasets import load_iris


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    parser.add_argument('--max_leaf_nodes', type=int, default=-1)

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])

    args = parser.parse_args()

    iris = load_iris()

    X = iris.data
    y = iris.target

    # Here we support a single hyperparameter, 'max_leaf_nodes'. Note that you can add as many
    # as your training my require in the ArgumentParser above.
    max_leaf_nodes = args.max_leaf_nodes

    # Now use scikit-learn's decision tree classifier to train the model.
    clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    clf = clf.fit(X, y)

    # Print the coefficients of the trained classifier, and save the coefficients
    joblib.dump(clf, os.path.join(args.model_dir, "model.joblib"))


def model_fn(model_dir):
    """Deserialized and return fitted model

    Note that this should have the same name as the serialized model in the main method
    """
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

Overwriting train.py


In [5]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

role = sagemaker.get_execution_role()
session = sagemaker.Session()

In [7]:
%pip install whylogs[whylabs]==1.1.28

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 10.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.7/370.7 kB 5.8 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [65]:
%%writefile requirements.txt
whylogs[whylabs]==1.1.28

Overwriting requirements.txt


In [8]:
estimator = SKLearn(
    entry_point="train.py",
    source_dir=".",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.0-1",
)

In [9]:
inputs = {
    "train": session.upload_data("train.csv", key_prefix="data")
}

In [11]:
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-03-06-13-54-37-067


2023-03-06 13:54:37 Starting - Starting the training job...
2023-03-06 13:54:55 Starting - Preparing the instances for training......
2023-03-06 13:55:41 Downloading - Downloading input data...
2023-03-06 13:56:11 Training - Downloading the training image..2023-03-06 13:56:50,758 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-03-06 13:56:50,762 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-03-06 13:56:50,770 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-03-06 13:56:50,988 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.0/548.0 kB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.7/370.7 kB 57.8 MB/s eta 0:00:00
[notice] A new rele

In [35]:
from sagemaker.predictor import Predictor

sklearn_predictor_dev = estimator.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1,    
    # predictor_cls=Predictor(content_type='application/json')
    endpoint_name='iris-sklearn-endpoint-dev',
    entry_point='predict.py',
    source_dir='.',
    env={
        "WHYLABS_DEFAULT_ORG_ID": "org-PbEh34",
        "WHYLABS_DEFAULT_DATASET_ID": "model-6",
        "WHYLABS_API_KEY": "wWI8y4sj0T.uKnAw1epbZeVOPfmopYri6eShX7mrBEovGbBi2zqESFY7fp7etZJ5",
        "WHYLABS_API_ENDPOINT": "https://songbird.development.whylabsdev.com"
        # "WHYLABS_DEFAULT_ORG_ID": "org-fjx9Rz",
        # "WHYLABS_DEFAULT_DATASET_ID": "model-8",
        # "WHYLABS_API_KEY": "9O8I1hdbIK.eJoaRIc4HMRVwodRXDGQNiaivoLO7ipqVNZHJCBT6Pm23UfTXSJXa"
    } 
)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2023-03-06-16-07-01-311
INFO:sagemaker:Creating endpoint-config with name iris-sklearn-endpoint-dev
INFO:sagemaker:Creating endpoint with name iris-sklearn-endpoint-dev


------!

In [26]:
import numpy as np
import json

data = np.array([5.1, 3.5, 1.4, 0.2]).reshape(1, -1)
payload = json.dumps(data.tolist())

response = sklearn_predictor.predict(data=data)
print(response)

[0]


In [36]:
resp_list = []

for _ in range(100):
    data = np.array(np.random.random(4)) * 5
    response = sklearn_predictor_dev.predict(data=data.reshape(1, -1))
    resp_list.append(response)